## Import packages and initialize AOAI connection

In [18]:
import re
import os
import PyPDF2
import openai
import requests
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

API_KEY = os.getenv("OPENAI_API_EMBEDDING_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_EMBEDDING_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})

# print(r.text)


## Define the search engine and

In [19]:
# embeddings_engine = 'text-search-davinci-doc-001'
# search_engine = 'text-search-davinci-query-001'
embeddings_engine = 'text-search-curie-doc-001' 
search_engine = 'text-search-curie-query-001'
query_string = 'what ports and connectors does my surface have?' # 'What are potential regulatory actions that can result from an inspection?'
summarization_engine = "text-davinci-002"

## Read in .pdf

In [22]:
def read_pdf_into_df(pdf_file_path):
    # create a pdf file object
    pdfFileObj = open(pdf_file_path, 'rb') # rb means read binary mode 
    # create a pdf reader object
    pdfReader = PyPDF2.PdfReader(pdfFileObj) 
    # print the number of pages in pdf document 
    print('Number of pages in PDF: ', len(pdfReader.pages)) 

    # declare an empty dataframe with two columns: Page and Text (for page number and text of each page)  
    df = pd.DataFrame(columns=['page', 'text'])

    # loop through the number of pages in the document, get each page's text, write to a new row in dataframe  
    for i in range(0, len(pdfReader.pages)):      # for each page 

        pageObj = pdfReader.pages[i]     # get that page object from the pdf reader object using indexing  

        text = pageObj.extract_text()       # extract the text from that page object  

        df_temp = pd.DataFrame([[i+1,text]], columns=['page', 'text'])     # create temporary dataframe to hold information for each iteration (page number and text on that page)  

        # df = df.append(df_temp)            # append temporary dataframe to master dataframe at end of each iteration (loop through all pages of document)  
        df = pd.concat([df, df_temp], ignore_index=True)     # append temporary dataframe to master dataframe at end of each iteration (loop through all pages of document)

        print('Finished processing page %d' % (i+1))      # print statement so you can see what pages have been processed already while script is running 

    print('Finished processing all pages.')               # print statement when script has finished running 

    return df

In [20]:
# Path to download the PDF to run this exercise: 'https://download.microsoft.com/download/7/B/1/7B10C82E-F520-4080-8516-5CF0D803EEE0/surface-book-user-guide-EN.pdf'
surface_user_guide_path = 'C:/Users/ianadams/OneDrive - Microsoft/surface-book-user-guide-EN.pdf'
surface_user_guide_df = read_pdf_into_df(surface_user_guide_path)

# fda_7348_810_path = "C:/Users/ianadams/OneDrive - Microsoft/CP 7348.810 Sponsors and CROs_FINAL.pdf"
# surface_user_guide_df = read_pdf_into_df(fda_7348_810_path)

Number of pages in PDF:  41
Finished processing page 1
Finished processing page 2
Finished processing page 3
Finished processing page 4
Finished processing page 5
Finished processing page 6
Finished processing page 7
Finished processing page 8
Finished processing page 9
Finished processing page 10
Finished processing page 11
Finished processing page 12
Finished processing page 13
Finished processing page 14
Finished processing page 15
Finished processing page 16
Finished processing page 17
Finished processing page 18
Finished processing page 19
Finished processing page 20
Finished processing page 21
Finished processing page 22
Finished processing page 23
Finished processing page 24
Finished processing page 25
Finished processing page 26
Finished processing page 27
Finished processing page 28
Finished processing page 29
Finished processing page 30
Finished processing page 31
Finished processing page 32
Finished processing page 33
Finished processing page 34
Finished processing page 35
F

## Drop the cover and contents pages

In [21]:
rows_to_drop = list(range(0,2)) # Drops Cover and Copyright
# rows_to_drop = list(range(0,5)) # Drops Cover and Table of Contents
surface_user_guide_df = surface_user_guide_df.drop(rows_to_drop, axis=0)

## Normalize Text in the df

In [23]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

In [24]:
surface_user_guide_df['text'] = surface_user_guide_df['text'].apply(lambda x : normalize_text(x))
# surface_user_df

## Get a token count for each page

In [25]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
surface_user_guide_df['n_tokens'] = surface_user_guide_df["text"].apply(lambda x: len(tokenizer.encode(x)))
# surface_user_df = surface_user_df[surface_user_df.n_tokens<2000]
len(surface_user_guide_df)
surface_user_guide_df

,page,text,n_tokens
2,3,© 201 6 Microsoft Page iii Contents Meet Surfa...,346
3,4,© 201 6 Microsoft Page iv BROWSING TIPS .........,379
4,5,© 201 6 Microsoft Page v Audio problems .........,322
5,6,© 201 6 Microsoft Page 1 Meet Surface Book Get...,163
6,7,© 201 6 Microsoft Page 2 The 10 -point multi -...,274
7,8,© 201 6 Microsoft Page 3 Ports and connectors ...,172
8,9,© 201 6 Microsoft Page 4 Apps Surface Book com...,280
9,10,© 201 6 Microsoft Page 5 Set up Windows Press ...,198
10,11,© 201 6 Microsoft Page 6 With the Clipboard de...,151
11,12,© 201 6 Microsoft Page 7 Note: Surface Book us...,235


## Get the embeddings for each page

In [37]:
surface_user_guide_df['davinci_search'] = surface_user_guide_df["text"].apply(lambda x : get_embedding(x, engine = embeddings_engine))
surface_user_guide_df

,page,text,n_tokens,davinci_search,similarities
2,3,"© 201 6 Microsoft Page iii Contents Meet Surface Book ......................................................................... 1 SURFACE BOOK FEATURES ..................................................................................... 1 Set up y our Surface Book .................................................................... 4 CHARGE YOUR SURFACE BOOK ................................................................................. 4 SET UP WINDOWS ........................................................................................... 5 CHOOSE A MODE TO WORK IN ................................................................................. 5 The basics ................................................................................ 7 POWER AND CHARGING ....................................................................................... 7 Check the battery level .................................................................................. 7 Making your battery last ................................................................................. 8 POWE R STATES : ON, OFF, SLEE AND RESTART ..................................................................... 8 TOUC KEYBOAR PEN, AND MOUSE ............................................................................ 9 Touchscreen ............................................................................................ 9 Keyboard .............................................................................................. 9 Touchpad or mouse .................................................................................... 10 Surface Pen (Surface Pro 4 version) ...................................................................... 11 Accounts and signing in .................................................................... 11 FIRST ACCOUNT ON YOUR SURFACE ............................................................................ 11 SIGN IN TO YOUR SURFACE ................................................................................... 11 USE WINDOWS HELLO ....................................................................................... 12 CHANGE YOUR PASSWOR PICTUR AND OTHER ACCOUNT S ETTINGS ................................................. 12 ADD AN ACCOUNT TO YOU R SURFACE ........................................................................... 12 CREATE A CHILD ACCOUNT ON YOUR SURFACE .................................................................... 13 Get to know Windows 10 ................................................................... 13 GO TO START .............................................................................................. 13 ACTION CENTER ............................................................................................ 14 SEARCH ................................................................................................... 14 TASK VIEW AND VIRTUAL DESKTOPS ............................................................................ 14 SETTINGS IN WINDOWS 10 ................................................................................... 15 Keep y our Surface Book up to date ........................................................... 16 Browse the Internet ........................................................................ 16",346,"[0.0038073223549872637, -0.0022003506310284138, 0.008025622926652431, 0.027152281254529953, -0.02146323211491108, -0.01151663064956665, -0.008824490942060947, -0.008085654117166996, 0.010002014227211475, 0.0029368794057518244, -0.0320470817387104, -0.01022366527467966, 0.0016485314117744565, -0.004880945663899183, -0.006039997097104788, -0.008007152006030083, -0.015395527705550194, 0.011812165379524231, -0.013234428130090237, 0.012615651823580265, 0.021574057638645172, 0.008990730158984661, -0.0005148774362169206, -0.00258593144826591, 0.024123046547174454, -0.0004848621611017734, 0.025896256789565086, -

## Define our search and execute

In [27]:
# search through the reviews for a specific product
def search_docs(df, user_query, search_engine, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine=search_engine
    )
    df["similarities"] = df.davinci_search.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print == True:
        display(res)
    return res


In [28]:
search_results = search_docs(surface_user_guide_df, query_string, search_engine, top_n=4)

,page,text,n_tokens,davinci_search,similarities
7,8,© 201 6 Microsoft Page 3 Ports and connectors ...,172,"[0.0008697164594195783, -0.008254462853074074,...",0.454285
28,29,© 201 6 Microsoft Page 24 Connect devices and ...,331,"[0.01547200046479702, -0.009754086844623089, 0...",0.406003
30,31,© 201 6 Microsoft Page 26 DisplayPort or HDMI ...,361,"[0.010627176612615585, -0.006943523418158293, ...",0.397813
29,30,© 201 6 Microsoft Page 25 Without external spe...,445,"[3.454791658441536e-05, 0.007500051986426115, ...",0.396857


In [29]:
search_results.reset_index(inplace=True)
search_results

,index,page,text,n_tokens,davinci_search,similarities
0,7,8,© 201 6 Microsoft Page 3 Ports and connectors ...,172,"[0.0008697164594195783, -0.008254462853074074,...",0.454285
1,28,29,© 201 6 Microsoft Page 24 Connect devices and ...,331,"[0.01547200046479702, -0.009754086844623089, 0...",0.406003
2,30,31,© 201 6 Microsoft Page 26 DisplayPort or HDMI ...,361,"[0.010627176612615585, -0.006943523418158293, ...",0.397813
3,29,30,© 201 6 Microsoft Page 25 Without external spe...,445,"[3.454791658441536e-05, 0.007500051986426115, ...",0.396857


## Pass text from top 3 results to AOAI for summarization

In [30]:
prompt_engineering = 'You must summarize the results of the ----SEARCH RESULTS---- section in a way that best answers the query listed in the ----USER QUERY--- section \n \n'
def create_final_prompt(df, user_query, prompt_engineering):
    res = prompt_engineering + '----USER QUERY----\n' + user_query + '\n\n' + '----SEARCH RESULTS----\n'

    for i in range(3):
        res += df['text'][i] + '\n'
    
    print(res)
    
    return res

In [31]:
final_prompt = create_final_prompt(search_results, query_string, prompt_engineering)

You must summarize the results of the ----SEARCH RESULTS---- section in a way that best answers the query listed in the ----USER QUERY--- section 
 
----USER QUERY----
what ports and connectors does my surface have?

----SEARCH RESULTS----
© 201 6 Microsoft Page 3 Ports and connectors Surface Book has the ports you expect from a full -feature laptop. Two full-size USB 3.0 ports Connect a USB accessory like a mouse, printer, Ethernet adapter, USB drive, or smartphone. SD card slot Use the full -size SD card slot with an SD card (sold separately) for extra storage and transferring files. Mini DisplayPort version 1.2a Share what’s on your Surface screen by connecting it to an HDTV, monitor, or projector. (Video adapters are sold separ ately.) 3.5 mm headset jack Plug in your favorite headset for a little more privacy when listening to music or conference calls. Software Windows 10 operating system Windows 10 provides a variety of options for entertainment and productivity whether you ’re 

In [32]:
def results_text_to_final_response(prompt, summarization_engine):
    # Passes the response from the question answering bot to the AOAI model

    # Format the output from the QA bot to include the signifier for a summarization from AOAI
    # bot_answer = answer + '\n\nTl;dr'

    # REVIEW: Print the length of the bot answer for reference - may want to cut off since summarization
    # doesn't work well with short answer.
    # print(len(bot_answer))

    # Submit the answer from the QA Bot to the AOAI model for summariation
    response = openai.Completion.create(
      engine=summarization_engine,
      prompt= prompt, 
      temperature=0.7,
      max_tokens=2048,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)

    return response

In [33]:
response = results_text_to_final_response(final_prompt, summarization_engine)

In [34]:
def print_formatted_response(open_ai_response):
        print("\n", "Azure Open AI response: ", open_ai_response['choices'][0]['text'])

print_formatted_response(response)


 Azure Open AI response:  
The Surface Book has two full-size USB 3.0 ports, an SD card slot, a Mini DisplayPort version 1.2a, and a 3.5 mm headset jack. The Surface Book can be transformed into a full desktop PC using the Surface Dock, which supports high -speed transfer of video, audio, and data, and has a compact design. The Surface Dock has an external power supply, rechargeable battery, USB ports, Mini DisplayPort, Ethernet port, and audio jack.
